In [5]:
from convokit import Corpus, download

# Download and load the GAP Corpus
gap_corpus = Corpus(filename=download("gap-corpus"))
print("GAP Corpus loaded successfully!")
gap_corpus.print_summary_stats()


Dataset already exists at /Users/sofiansyed/.convokit/saved-corpora/gap-corpus
GAP Corpus loaded successfully!
Number of Speakers: 84
Number of Utterances: 8009
Number of Conversations: 28


In [6]:
import pandas as pd
from convokit import Corpus, download

# Extracts each speaker's Ind_Lead score (self-perceived leadership).
# Maps it to a descriptive label (e.g., "Very High").

score_mapping = {
    1: "Very Low",
    2: "Low",
    3: "Moderate",
    4: "High",
    5: "Very High"
}


speakers_data = []
# Iterate over speaker objects using iter_speakers()
for speaker in gap_corpus.iter_speakers():
    # Get the leadership score from metadata and ensure it's an integer if possible
    score = speaker.meta.get("Ind_Lead", None)
    try:
        score = int(score)
    except (ValueError, TypeError):
        score = None

    # Map the numeric score to a descriptive label, if available
    score_label = score_mapping.get(score, "N/A") if score is not None else "N/A"
    
    conversation_id = speaker.id.split('.')[0]
    
    speakers_data.append({
        "Conversation": conversation_id,
        "Speaker": speaker.id,
        "   Numeric Leadership Score   ": score if score is not None else "N/A",
        "Descriptive Leadership": score_label
    })

df = pd.DataFrame(speakers_data)
df.sort_values(by=["Conversation", "Speaker"], inplace=True)
for conversation, group in df.groupby("Conversation"):
    print("="*50)
    print(f"Conversation {conversation}")
    print("="*50)
    print(group.to_string(index=False))
    print("\n")


Conversation 1
Conversation Speaker     Numeric Leadership Score    Descriptive Leadership
           1  1.Blue                               2                    Low
           1 1.Green                               4                   High
           1  1.Pink                               5              Very High


Conversation 10
Conversation   Speaker     Numeric Leadership Score    Descriptive Leadership
          10 10.Orange                               4                   High
          10   10.Pink                               3               Moderate


Conversation 11
Conversation  Speaker     Numeric Leadership Score    Descriptive Leadership
          11 11.Green                               5              Very High
          11  11.Pink                               4                   High


Conversation 12
Conversation   Speaker     Numeric Leadership Score    Descriptive Leadership
          12   12.Blue                               3               Moderate
      

In [7]:
from collections import defaultdict
from convokit import Corpus, download
from convokit.coordination import Coordination
import numpy as np

# Uses ConvoKit’s Coordination transformer to measure linguistic coordination (e.g., matching pronouns, articles).
# Computes:
# Raw coordination scores
# 3 weighted versions (Eq1, Eq2, Eq3)
# Average of the 3 equations
# A final score (average of raw + weighted)
# Groups speakers by conversation.
# Prints all these values in a  table and identifies the highest scorer per group.

coord = Coordination(coordination_attribute_name='coord')
# Fit the transformer on the corpus and transform it.
# This annotates each speaker in the corpus with coordination scores stored in their meta.
gap_corpus = coord.fit_transform(gap_corpus)

# Equation 2: "Communication Influence" weighting (emphasizes auxverb, conj, adverb)
weights_eq2 = {
    "article": 1.0,
    "auxverb": 1.5,
    "conj": 2.0,
    "adverb": 2.0,
    "ppron": 1.0,
    "ipron": 1.0,
    "preps": 0.5,
    "quant": 0.5
}
# Equation 3: "Structural Emphasis" weighting (emphasizes conj and preps)
weights_eq3 = {
    "article": 0.5,
    "auxverb": 1.0,
    "conj": 2.5,
    "adverb": 1.0,
    "ppron": 0.5,
    "ipron": 0.5,
    "preps": 2.0,
    "quant": 0.5
}

# Function to compute weighted averages from a marker dictionary.
def compute_target_scores(marker_dict):
    # Collect numeric values.
    values = [v for v in marker_dict.values() if isinstance(v, (int, float))]
    if not values:
        return None, None, None
    # Equation 1: Equal weighting -> simple average.
    eq1 = np.mean(values)
    
    # Equation 2: Communication Influence weighting.
    num2, den2 = 0.0, 0.0
    for marker, value in marker_dict.items():
        if isinstance(value, (int, float)) and marker in weights_eq2:
            num2 += weights_eq2[marker] * value
            den2 += weights_eq2[marker]
    eq2 = num2 / den2 if den2 != 0 else None

    # Equation 3: Structural Emphasis weighting.
    num3, den3 = 0.0, 0.0
    for marker, value in marker_dict.items():
        if isinstance(value, (int, float)) and marker in weights_eq3:
            num3 += weights_eq3[marker] * value
            den3 += weights_eq3[marker]
    eq3 = num3 / den3 if den3 != 0 else None

    return eq1, eq2, eq3

conversation_groups = defaultdict(list)

# Process each speaker in the corpus.
for speaker in gap_corpus.iter_speakers():
    # Extract conversation group from speaker id (e.g., "1.Pink" -> "1")
    conv_group = speaker.id.split('.')[0]
    # Initialize lists to hold target-level scores for aggregation.
    raw_list = []   # raw final score: sum of all pairwise scores
    eq1_list = []   # Equation 1 scores per target
    eq2_list = []   # Equation 2 scores per target
    eq3_list = []   # Equation 3 scores per target
    
    # Get the coordination data for this speaker.
    coordination_data = speaker.meta.get('coord', {})
    
    for target, score in coordination_data.items():
        # If score is a dictionary, check if an 'overall' key exists.
        if isinstance(score, dict):
            if 'overall' in score:
                overall_val = score['overall']
                raw_list.append(overall_val)
                eq1_list.append(overall_val)
                eq2_list.append(overall_val)
                eq3_list.append(overall_val)
            else:
                res = compute_target_scores(score)
                if res[0] is not None:
                    eq1_list.append(res[0])
                    eq2_list.append(res[1])
                    eq3_list.append(res[2])
                    raw_list.append(np.sum([v for v in score.values() if isinstance(v, (int, float))]))
        elif isinstance(score, (int, float)):
            raw_list.append(score)
            eq1_list.append(score)
            eq2_list.append(score)
            eq3_list.append(score)
    
    # Compute aggregated scores per speaker.
    aggregated_eq1 = np.mean(eq1_list) if eq1_list else None
    aggregated_eq2 = np.mean(eq2_list) if eq2_list else None
    aggregated_eq3 = np.mean(eq3_list) if eq3_list else None
    aggregated_raw = np.sum(raw_list) if raw_list else None
    
    # Compute the average of the three equation scores.
    avg_weighted = None
    if aggregated_eq1 is not None and aggregated_eq2 is not None and aggregated_eq3 is not None:
        avg_weighted = (aggregated_eq1 + aggregated_eq2 + aggregated_eq3) / 3

    # Final score is 50% from the aggregated raw score and 50% from the average of the three equation scores.
    final_score = None
    if aggregated_raw is not None and avg_weighted is not None:
        final_score = 0.5 * aggregated_raw + 0.5 * avg_weighted
    
    speaker_aggregated = {
        "id": speaker.id,
        "raw_final": aggregated_raw,
        "eq1": aggregated_eq1,
        "eq2": aggregated_eq2,
        "eq3": aggregated_eq3,
        "avg_weighted": avg_weighted,
        "final": final_score
    }
    
    # Append the speaker data object to its conversation group.
    conversation_groups[conv_group].append(speaker_aggregated)

# STEP 3: Sort by Conversation Group and Print
print("\n=== Detailed Coordination Scores, Final Scores, and Winners by Conversation Group ===\n")
for conv in sorted(conversation_groups.keys(), key=lambda x: int(x)):
    print("=" * 100)
    print(f"Conversation Group: {conv}")
    print("=" * 100)
    header = (f"{'Speaker':<10} {'Raw Final':>10}   {'Eq1 (Equal)':>12}   "
              f"{'Eq2 (Comm Inf.)':>16}   {'Eq3 (Struct Emph.)':>18}   "
              f"{'Avg Weighted':>14}   {'Final Score':>12}")
    print(header)
    print("-" * len(header))
    
    winner = None
    highest_final = -np.inf
    for s_data in conversation_groups[conv]:
        s_id = s_data["id"]
        raw_final = s_data["raw_final"]
        eq1 = s_data["eq1"]
        eq2 = s_data["eq2"]
        eq3 = s_data["eq3"]
        avg_weighted = s_data["avg_weighted"]
        final = s_data["final"]
        
        print(f"{s_id:<10} {raw_final:10.4f}   {eq1:12.4f}   {eq2:16.4f}   {eq3:18.4f}   {avg_weighted:14.4f}   {final:12.4f}")
        
        if final is not None and final > highest_final:
            highest_final = final
            winner = s_id
            
    print("-" * len(header))
    print(f"Winner (highest Final Score): {winner} with a score of {highest_final:.4f}")
    print("\n")



=== Detailed Coordination Scores, Final Scores, and Winners by Conversation Group ===

Conversation Group: 1
Speaker     Raw Final    Eq1 (Equal)    Eq2 (Comm Inf.)   Eq3 (Struct Emph.)     Avg Weighted    Final Score
------------------------------------------------------------------------------------------------------------
1.Pink         0.8485         0.0530             0.0607               0.0547           0.0561         0.4523
1.Blue        -0.4588        -0.0278            -0.0302              -0.0329          -0.0303        -0.2445
1.Green        2.0156         0.1440             0.1512               0.0906           0.1286         1.0721
------------------------------------------------------------------------------------------------------------
Winner (highest Final Score): 1.Green with a score of 1.0721


Conversation Group: 2
Speaker     Raw Final    Eq1 (Equal)    Eq2 (Comm Inf.)   Eq3 (Struct Emph.)     Avg Weighted    Final Score
------------------------------------------

In [8]:
# Compare Winners for Each Coordination Scheme with Self-Perceived Leader (Allowing Ties)

# Build a dictionary mapping speaker id to their self-perceived leadership score.
self_lead_dict = {}
for speaker in gap_corpus.iter_speakers():
    try:
        self_lead_dict[speaker.id] = float(speaker.meta.get("Ind_Lead", 0))
    except (TypeError, ValueError):
        self_lead_dict[speaker.id] = 0

# Define the coordination schemes (columns) to evaluate.
schemes = ["raw_final", "eq1", "eq2", "eq3", "avg_weighted", "final"]

scheme_match_counts = {scheme: 0 for scheme in schemes}
total_groups = 0

print("\n=== Leader Comparison for Each Coordination Scheme by Conversation Group ===\n")
for conv in sorted(conversation_groups.keys(), key=lambda x: int(x)):
    total_groups += 1
    group = conversation_groups[conv]
    
    # Determine the self-perceived leader(s) for this group (allowing ties).
    highest_self = -np.inf
    self_winners = []
    for s_data in group:
        s_id = s_data["id"]
        self_rating = self_lead_dict.get(s_id, 0)
        if self_rating > highest_self:
            highest_self = self_rating
            self_winners = [s_id]
        elif self_rating == highest_self:
            self_winners.append(s_id)
    
    print("=" * 100)
    print(f"Conversation Group: {conv}")
    print(f"Self-Perceived Leader(s): {', '.join(self_winners)} (Ind_Lead: {highest_self:.4f})")
    print("=" * 100)
    
    # For each coordination scheme, determine the winner(s) and compare.
    for scheme in schemes:
        highest_value = -np.inf
        scheme_winners = []
        for s_data in group:
            score = s_data.get(scheme)
            if score is not None:
                if score > highest_value:
                    highest_value = score
                    scheme_winners = [s_data["id"]]
                elif score == highest_value:
                    scheme_winners.append(s_data["id"])
        
        # Check if any of the scheme winners match any of the self-perceived winners.
        match = any(winner in self_winners for winner in scheme_winners)
        if match:
            scheme_match_counts[scheme] += 1
        
        print(f"Scheme '{scheme}':")
        print(f"  Model Winner(s): {', '.join(scheme_winners)} (Score: {highest_value:.4f})")
        print(f"  Match with self-perceived leader(s): {'Yes' if match else 'No'}")
        print("-" * 100)
    print("\n")




=== Leader Comparison for Each Coordination Scheme by Conversation Group ===

Conversation Group: 1
Self-Perceived Leader(s): 1.Pink (Ind_Lead: 5.0000)
Scheme 'raw_final':
  Model Winner(s): 1.Green (Score: 2.0156)
  Match with self-perceived leader(s): No
----------------------------------------------------------------------------------------------------
Scheme 'eq1':
  Model Winner(s): 1.Green (Score: 0.1440)
  Match with self-perceived leader(s): No
----------------------------------------------------------------------------------------------------
Scheme 'eq2':
  Model Winner(s): 1.Green (Score: 0.1512)
  Match with self-perceived leader(s): No
----------------------------------------------------------------------------------------------------
Scheme 'eq3':
  Model Winner(s): 1.Green (Score: 0.0906)
  Match with self-perceived leader(s): No
----------------------------------------------------------------------------------------------------
Scheme 'avg_weighted':
  Model Winner(s):

In [9]:
print("=== Overall Accuracy per Coordination Scheme ===")
for scheme in schemes:
    accuracy = (scheme_match_counts[scheme] / total_groups) * 100 if total_groups > 0 else 0
    print(f"{scheme:<12}: {accuracy:6.2f}% ({scheme_match_counts[scheme]} out of {total_groups} groups)")


=== Overall Accuracy per Coordination Scheme ===
raw_final   :  35.71% (10 out of 28 groups)
eq1         :  39.29% (11 out of 28 groups)
eq2         :  32.14% (9 out of 28 groups)
eq3         :  39.29% (11 out of 28 groups)
avg_weighted:  32.14% (9 out of 28 groups)
final       :  35.71% (10 out of 28 groups)
